## For saving track files from dataframes

In [47]:
'''
This code generates a dataset of biased random walks that is stored in Pandas DataFrames.
'''

# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pandas as pd


# Constants
START_TIME = 0.0 #do not modify
END_TIME = 20.0 #duration of tracking 'experiment'
TIME_INCREMENT = 0.1 #time step size
STEP_NUMBER = int(END_TIME/TIME_INCREMENT) #do not modify

WORLD_SIZE_X = 10.0 #world width 
WORLD_SIZE_Y = 10.0 #world height

START_POSITION_RANDOM = True #toggle whether start position is random
START_POSITION = (5.0, 5.0) #start position if not random (set to centre of world, may be modified)

TOTAL_SPEED = 1.2 #assumed to be constant property of bacterium in given environment

TUMBLE_THRESHOLD = 0.6 #threshold used by reorientation function; higher threshold makes reorientation less likely

REPEAT_NUMBER = 1000 #number of tracks to be obtained


# Helper functions
def test_out_of_bounds(x, y, velocity_x, velocity_y):
    '''
    Test whether the cell has reached the walls of the world and reset velocity components to make cell bounce off 
    wall if appropriate. Collisions are assumed to be perfectly elastic with the wall infinitely heavy.
    '''
    if x >= WORLD_SIZE_X or x <= 0:
        velocity_x = -velocity_x
    if y >= WORLD_SIZE_Y or y <= 0:
        velocity_y = -velocity_y
    return (velocity_x, velocity_y)

def attractant_distribution(x, y):
    '''
    Describes the distribution of an attractant. This can be modified by the user.
    Note the function should not have zero values within the range of coordinates included in the world.
    '''
    #attractant_concentration = 0.15*x**2 - 0.15*y**2 + 0.1
    attractant_concentration = 10
    return attractant_concentration

def set_speed():
    '''
    Sets x- and y-components of the velocity to random values.
    The vector magnitude (TOTAL_SPEED) is kept constant.
    '''
    velocity_x = random.uniform(-TOTAL_SPEED, TOTAL_SPEED) #randomise x-component of velocity
    velocity_y = (TOTAL_SPEED**2 - velocity_x**2) #use formula for vector magnitude
    random_orientation_y = random.uniform(-1,1) #magnitude of velocity_y is fixed when velocity_x is fixed, but not sign
    if random_orientation_y >= 0.0: #use random number to determine sign of y-component
        velocity_y = -velocity_y
    return (velocity_x, velocity_y)

def set_start_position():
    '''
    Generate a random start position for each track
    '''
    x = random.uniform(0.0,10.0)
    y = random.uniform(0.0,10.0)
    return (x,y)

def update_position(x, y, velocity_x, velocity_y):
    '''
    Update the position of the cell with each time step.
    '''
    x = x + velocity_x * TIME_INCREMENT
    y = y + velocity_y * TIME_INCREMENT
    return (x, y)

def reorientation(time, time_last_tumble, x, y):
    '''
    Determines whether the cell should reorient at a given time (returns a Boolean).
    Reorientation probability depends on the time since last tumble and the attractant concentration.
    A random component is added using a random number generator (motor switching is partly stochastic).
    '''
    attractant_concentration = attractant_distribution(x, y)
    
    time_since_last_tumble = time - time_last_tumble
    random_number = random.uniform(0.0,5.0)
    time_dependent_random_number = time_since_last_tumble * random_number
    
    return (time_dependent_random_number / attractant_concentration) > TUMBLE_THRESHOLD

def get_track():
    '''
    Generates a random track using the specified constants. 
    A random start position may be chosen, or this may be specified as a constant
    '''
    cell_location_x = np.zeros(STEP_NUMBER) #set up empty array to store track x-coordinates
    cell_location_y = np.zeros(STEP_NUMBER) #set up empty array to store track y-coordinates
    run_length = [] #set up empty list in which to store run lengths (in units of time)
    
    time = START_TIME
    time_last_tumble = START_TIME
    (velocity_x, velocity_y) = set_speed()
    
    if START_POSITION_RANDOM:
        start_position = set_start_position()
        (x,y) = start_position
    else:
        (x,y) = START_POSITION
    
    for i in range(STEP_NUMBER):
        (velocity_x, velocity_y) = test_out_of_bounds(x,y, velocity_x, velocity_y)
        (x, y) = update_position(x, y, velocity_x, velocity_y)
        cell_location_x[i] = x
        cell_location_y[i] = y
        time = time + TIME_INCREMENT
        if reorientation(time, time_last_tumble, x, y):
            (velocity_x, velocity_y) = set_speed()
            run_length.append(time-time_last_tumble)
            time_last_tumble = time
    end_position = (x,y)
            
    return cell_location_x, cell_location_y, run_length, start_position, end_position


#main function (obtain database for multiple tracks)
'''
Generates and stores a number (REPEAT_NUMBER) of random tracks produced by get_track().
Results are stored in lists. Associated track numbers are stored in separate lists.
'''
run_length_database = [] #create empty lists that will become dataframe columns
run_length_track_numbers_database = []
x_coordinate_database = []
y_coordinate_database = []
coordinate_track_numbers_database = []
x_displacement_database = []
y_displacement_database = []
displacement_track_numbers_database = []

for i in range(REPEAT_NUMBER):
    (cell_location_x, cell_location_y, run_length, start_position, end_position) = get_track()
    
    run_length_database.extend(run_length) #add new values to future columns
    x_coordinate_database.extend(cell_location_x)
    y_coordinate_database.extend(cell_location_y)
    x_displacement_database.append(end_position[0]-start_position[0])
    y_displacement_database.append(end_position[1]-start_position[1])
    
    run_length_track_number = [i]*len(run_length) #generate lists of track numbers to associate with values in columns
    run_length_track_numbers_database.extend(run_length_track_number)
    coordinate_track_number = [i]*len(cell_location_x)
    coordinate_track_numbers_database.extend(coordinate_track_number)
    displacement_track_numbers_database.append(i)

    
#dataframe creation
'''
Stores the track coordinates and relevant parameters in Pandas dataframes.
'''
df_coordinates = pd.DataFrame([coordinate_track_numbers_database, x_coordinate_database, y_coordinate_database])
df_coordinates = df_coordinates.transpose()
df_coordinates.columns = ['track_number_coordinate','x_coordinate', 'y_coordinate']

df_run_lengths = pd.DataFrame([run_length_track_numbers_database, run_length_database])
df_run_lengths = df_run_lengths.transpose()
df_run_lengths.columns = ['track_number_run', 'run_length']

df_displacements = pd.DataFrame([displacement_track_numbers_database, x_displacement_database, y_displacement_database])
df_displacements = df_displacements.transpose()
df_displacements.columns = ['track_number_displacement', 'x_displacement', 'y_displacement']

In [48]:
df_total = pd.concat([df_coordinates, df_displacements, df_run_lengths], axis=1)
df_total.head()

,track_number_coordinate,x_coordinate,y_coordinate,track_number_displacement,x_displacement,y_displacement,track_number_run,run_length
0,0.0,3.713375,2.422174,0.0,1.532027,-0.405920,0.0,2.5
1,0.0,3.793005,2.341584,1.0,1.216321,1.862701,0.0,1.7
2,0.0,3.872635,2.260994,2.0,-2.996871,1.262521,0.0,1.7
3,0.0,3.952265,2.180403,3.0,-2.912884,0.810148,0.0,1.9
4,0.0,4.031895,2.099813,4.0,-2.327805,-0.906140,0.0,1.7


In [49]:
df_total.to_csv('attractantisten.csv', index=False)

In [42]:
df_total.to_csv('..\Pandas_files\\attractantisten')